conda install openpyxl

In [46]:
import pandas as pd  
from datetime import datetime, timedelta
import re
import os
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words

In [47]:
#กำหนด input file / ชื่อ output file
input='Biore UV Aqua Rich Watery Essence_20240924_192601.csv'
output='Biore UV Aqua Rich Watery Essence'

In [48]:
def convert_date(date_str, file_date):
    # แปลงวันที่จากชื่อไฟล์
    file_date = datetime.strptime(file_date, '%Y%m%d')
    
    # กรณีที่มีคำว่า 'ก่อน'
    if 'ก่อน' in date_str:
        value, unit = date_str.split()[:2]
        value = int(value)
        
        if unit in ['วินาทีก่อน', 'นาทีก่อน', 'ชั่วโมงก่อน']:
            return file_date.strftime('%Y%m%d')
        elif unit == 'วันก่อน':
            return (file_date - timedelta(days=value)).strftime('%Y-%m-%d')
        elif unit == 'สัปดาห์ก่อน':
            return (file_date - timedelta(weeks=value)).strftime('%Y-%m-%d')
    
    month_dict = {
        'ม.ค.': '01', 'ก.พ.': '02', 'มี.ค.': '03', 'เม.ย.': '04',
        'พ.ค.': '05', 'มิ.ย.': '06', 'ก.ค.': '07', 'ส.ค.': '08',
        'ก.ย.': '09', 'ต.ค.': '10', 'พ.ย.': '11', 'ธ.ค.': '12'
    }
    
    parts = date_str.split()
    if len(parts) == 3 and parts[1] in month_dict:
        day, month, year = parts
        return f"{year}-{month_dict[month]}-{int(day):02d}"
    
    return date_str

def process_dataframe(file_path):
    # อ่านไฟล์ CSV
    df = pd.read_csv(file_path)
    
    # แยกวันที่จากชื่อไฟล์
    file_date = file_path.split('_')[-2]
    
    # แปลงคอลัมน์ date
    df['date'] = df['date'].apply(lambda x: convert_date(x, file_date))
    
    return df


In [ ]:
df = process_dataframe(input)
print(df)

In [ ]:
df.dtypes

In [50]:
# ฟังก์ชันลบข้อมูลที่ไม่ใช่ภาษาไทย
def clean_review(text):
    # ลบอีโมจิ, ภาษาอังกฤษ, สัญลักษณ์พิเศษ, และช่องว่างเกินจำเป็น โดยคงไว้เฉพาะตัวเลขและภาษาไทย
    cleaned_text = re.sub(r'[^\u0E00-\u0E7F0-9\s]', '', text)  # คงภาษาไทยและตัวเลข
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # ลบช่องว่างเกินและช่องว่างหน้าหลัง
    return cleaned_text

# ใช้ฟังก์ชัน clean_review กับคอลัมน์ review_text
df['review_text'] = df['review_text'].apply(lambda x: clean_review(str(x)))

In [ ]:
df

In [52]:
# ลบการขึ้นบรรทัดใหม่ใน review_text และทำให้ข้อความอยู่ในบรรทัดเดียวกัน
df['review_text'] = df['review_text'].apply(lambda x: str(x).replace('\n', ' ').strip())

In [ ]:
# แก้ไขคำที่ผิดปกติ หรือเขียนผิดซ้ำบ่อยๆ
# โหลดฐานข้อมูลคำพื้นฐาน
word_list = set(thai_words())

def is_meaningful(word):
    # เพิ่มการตรวจสอบคำที่มีรูปแบบพิเศษ
    special_patterns = [
        r'^[ก-ฮ]+$',  # คำภาษาไทยทั่วไป
        r'^[ก-ฮ]+ๆ$',  # คำซ้ำ เช่น ดีๆ เร็วๆ
        r'^[ก-ฮ]+[ุึ][ปบ]$',  # คำลงท้ายด้วย ุป, ึป, ุบ, ึบ เช่น ปุ๊ป, ปั๊บ
        r'^แพ[็ค]$',  # คำว่า แพ็ค หรือ แพค
        r'^[ก-ฮ]{1,2}่[ก-ฮ]$',  # คำสั้นๆ ที่มีวรรณยุกต์ เช่น ค่ะ ไว
        r'^[0-9๐-๙]+$',  # ตัวเลข
        r'^[a-zA-Z]+$',  # คำภาษาอังกฤษ
    ]
    
    return (word in word_list or
            any(re.match(pattern, word) for pattern in special_patterns))

def clean_text(text):
    # แยกคำ
    words = word_tokenize(text)
    
    # แทนที่คำที่ไม่มีความหมายด้วยช่องว่าง
    cleaned_words = [word if is_meaningful(word) else ' ' for word in words]
    
    # รวมคำกลับเป็นประโยค
    cleaned_text = ''.join(cleaned_words)
    
    # ลบช่องว่างซ้ำ
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # ลบช่องว่างที่ไม่จำเป็นตอนต้นและท้ายข้อความ
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

In [54]:
df['review_text'] = df['review_text'].apply(clean_text)

In [55]:
# ลบแถวที่ review_text ว่าง (ที่ไม่มีข้อมูลภาษาไทยหรือเป็นช่องว่าง)
df_cleaned = df[df['review_text'].astype(bool)].copy()

In [ ]:
df_cleaned

In [57]:
# รีเซ็ตเลขลำดับคอลัมน์ 'no' ใน DataFrame ที่ผ่านการตรวจสอบ
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['no'] = df_cleaned.index + 1

In [ ]:
df_cleaned

In [81]:
if not os.path.exists('CleanDataCSV'):
    os.makedirs('CleanDataCSV')
if not os.path.exists('CleanDataExel'):
    os.makedirs('CleanDataExel')

In [27]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'CleanDataCSV/{output}.csv'
df_cleaned.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'CleanDataExel/{output}.xlsx'
df_cleaned.to_excel(excel_file, index=False)